In [1]:
from multiprocessing import Pool, cpu_count
import gc; gc.enable()

import pandas as pd
import numpy as np
from sklearn import *
from datetime import datetime
import sklearn
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

<h2>Feature Engineering

<h3>Der Adaptive Domain Labeling Ansatz setzt ab hier auf. Dateien werden als train.csv und test.csv in SQLite importiert und manipuliert und anschließend in folgenden Dateien bereitgestellt

In [4]:
# Einlesen der Feature Engineerten Daten

## Fertige Feature export Datei #1 einlesen
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\export_wsdm.csv")

## Labels einlesen
wsdm_dataset_labels = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\train_v2.csv")

In [5]:
# Merge die Features der Trainigsdaten
wsdm_dataset = pd.merge(wsdm_dataset, wsdm_dataset_labels, on='msno', how='inner')

In [6]:
#Spalten an Anfang verschieben train
first_column = wsdm_dataset.pop('msno')
second_column = wsdm_dataset.pop('is_churn')
wsdm_dataset.insert(0, 'msno', first_column)
wsdm_dataset.insert(1, 'is_churn', second_column)

In [14]:
# reference: https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e
# reference: https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html
# reference: https://numpy.org/doc/stable/reference/generated/numpy.finfo.html

def datatype_changer(dataset):
    # iterating through all the columns in the dataframe
    for col in dataset.columns:
        # getting column's datatype
        col_type = dataset[col].dtype
        
        # checking if datatype of column is 'object' or not
        # if column type is not object
        if (col_type == int) or (col_type == float):
            # getting minimum value of a column
            min_val = dataset[col].min()
            # getting maximum value of a column
            max_val = dataset[col].max()
            # checking whether the datatype contain first 3 characters as int or not, if int
            if str(col_type)[:3] == 'int':
                # cheking the minimal and maximal value for int8, int16, int32 and int64 in numpy
                if min_val > np.iinfo(np.int8).min and max_val < np.iinfo(np.int8).max:
                    dataset[col] = dataset[col].astype(np.int8)
                elif min_val > np.iinfo(np.int16).min and max_val < np.iinfo(np.int16).max:
                    dataset[col] = dataset[col].astype(np.int16)
                elif min_val > np.iinfo(np.int32).min and max_val < np.iinfo(np.int32).max:
                    dataset[col] = dataset[col].astype(np.int32)
                else:
                    dataset[col] = dataset[col].astype(np.int64)
            else:
                # if it is non int, which is ultimately float
                # cheking the minimal and maximal value for float16, float32 and float64 in numpy
                if min_val > np.finfo(np.float16).min and max_val < np.finfo(np.float16).max:
                    dataset[col] = dataset[col].astype(np.float16)
                elif min_val > np.finfo(np.float32).min and max_val < np.finfo(np.float32).max:
                    dataset[col] = dataset[col].astype(np.float32)
                else:
                    dataset[col] = dataset[col].astype(np.float64)
        else:
            # keeping rest of them to category datatype instead of object
            dataset[col] = dataset[col].astype('category')
            
    # returning head of the dataframe
    return dataset

In [8]:
#Doppelte Features entfernen
# wsdm_dataset = wsdm_dataset.loc[:,~wsdm_dataset.T.duplicated(keep='first')]

In [9]:
#Spalten mit Nullvalues entfernen
wsdm_dataset = wsdm_dataset.drop(['DiffMAXdateMAXdate'], axis=1, inplace=False)

In [10]:
## Datentypen korrigieren
wsdm_dataset = datatype_changer(wsdm_dataset)

In [11]:
wsdm_dataset.isnull().sum()

msno                    0
is_churn                0
AVGregistered_via       0
AVGpayment_plan_days    0
AVGgender               0
                       ..
MAXnum_unqstock         0
MAXnum_25stock          0
MAXnum_100stock         0
MAXtotal_secsstock      0
msno.1                  0
Length: 142, dtype: int64

In [12]:
# saving wsdm file
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\wsdm_FA_adaptive_dl.csv", index=False)

<h1>3. Prepare Data for Modelling

In [2]:
# loading wsdm dataset
wsdm_dataset = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\wsdm_FA_adaptive_dl.csv")

In [3]:
# saving actual train dataset
wsdm_dataset.to_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train.csv", index=False)

<h2> 3.2 Train Test Split

In [4]:
# loading actual wsdm data
wsdm_data = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train.csv")

In [5]:
# Load RAW Data
wsdm_dataset_raw = pd.read_csv(r"D:\Work_Masterarbeit\wsdm\preprocessed_2\train.csv")

In [6]:
# Merging with RAW Data
wsdm_dataset = pd.merge(wsdm_dataset_raw, wsdm_data, on='msno', how='left')

In [7]:
wsdm_dataset.shape

(16887877, 164)

In [8]:
## Sample 50% of Data due to memory restrictions
wsdm_data = wsdm_dataset.sample(frac=.5)

In [9]:
#Typkonvertierung
wsdm_data['transaction_date'] = wsdm_data['transaction_date'].str.replace('-', '').astype(float)
wsdm_data['date'] = wsdm_data['date'].str.replace('-', '').astype(float)
wsdm_data['registration_init_time'] = wsdm_data['registration_init_time'].str.replace('-', '').astype(float)
wsdm_data['membership_expire_date'] = wsdm_data['membership_expire_date'].str.replace('-', '').astype(float)

In [10]:
#Entfernen unnötiger Spalten
wsdm_data = wsdm_data.drop(['msno.1'], axis=1, inplace=False)

In [11]:
# https://stackoverflow.com/a/29651514
def normalize(df):
    result1 = df.copy()
    for feature_name in df.columns:
        if (str(feature_name) != str('msno') and str(feature_name)!=str('is_churn')):
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result1[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result1

wsdm_dataset = normalize(wsdm_data)

In [12]:
# checking for null values
wsdm_data.isnull().values.any()

False

In [15]:
wsdm_data = datatype_changer(wsdm_data)

In [16]:
# getting columns and converting them to list
cols = wsdm_data.columns.tolist()
# saving them for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\columns_incraw.npz", cols)

In [17]:
# replacing all nan and infinity to 0
wsdm_data = wsdm_data.replace(np.nan, 0, inplace=False)
wsdm_data = wsdm_data.replace(np.inf, 0, inplace=False)

In [18]:
wsdm_data.head()

,msno,is_churn_x,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,MAXcitystock,MAXbdstock,MAXpayment_method_idstock,MAXis_auto_renewstock,MAXplan_list_pricestock,MAXactual_amount_paidstock,MAXnum_unqstock,MAXnum_25stock,MAXnum_100stock,MAXtotal_secsstock
346838,ZjI4qXriuGSV5eYJF54Gu10/JQPe/AS5M4Uq2gvus5o=,1,1.0,28.0,0.0,9.0,20160404.0,41.0,30.0,99.0,...,1.0,28.0,41.0,1.0,149.0,149.0,60.0,7.0,70.0,18512.0
3718453,sABfevyWzKyaoaP0lqGKtUt/MeT/9KbkSslCUfuTBhY=,0,12.0,25.0,1.0,7.0,20151112.0,41.0,30.0,99.0,...,12.0,25.0,41.0,1.0,99.0,99.0,23.0,15.0,34.0,9928.0
15769986,0mCI2cU8H0dgrIX86iOVd22/v1OeNnLrlP0Xs91aCOw=,0,1.0,28.0,0.0,7.0,20121218.0,41.0,30.0,99.0,...,1.0,28.0,41.0,1.0,129.0,129.0,34.0,5.0,48.0,9064.0
9347708,3HIXNrdUEna+pZNjpGyGwP+NOaEntix/9Ei7ttExBr4=,0,15.0,27.0,1.0,4.0,20160804.0,37.0,30.0,149.0,...,15.0,27.0,37.0,1.0,149.0,149.0,63.0,13.0,72.0,18992.0
6588668,SKzSHJ88cg+NQOHsCD/5m14ilPf8mcRMRU6rbYBHfgw=,0,1.0,28.0,0.0,3.0,20130504.0,38.0,30.0,149.0,...,1.0,28.0,38.0,0.0,149.0,149.0,27.0,4.0,12.0,3682.0


In [19]:
# getting labels for y
labels = wsdm_data['is_churn_x'].values
labels

[1, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0]
Length: 8443938
Categories (2, int64): [0, 1]

In [20]:
# getting labels for X
data = wsdm_data.drop(['msno', 'is_churn_x','is_churn_y'], axis=1, inplace=False)
data

,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,...,MAXcitystock,MAXbdstock,MAXpayment_method_idstock,MAXis_auto_renewstock,MAXplan_list_pricestock,MAXactual_amount_paidstock,MAXnum_unqstock,MAXnum_25stock,MAXnum_100stock,MAXtotal_secsstock
346838,1.0,28.0,0.0,9.0,20160404.0,41.0,30.0,99.0,99.0,1.0,...,1.0,28.0,41.0,1.0,149.0,149.0,60.0,7.0,70.0,18512.0
3718453,12.0,25.0,1.0,7.0,20151112.0,41.0,30.0,99.0,99.0,1.0,...,12.0,25.0,41.0,1.0,99.0,99.0,23.0,15.0,34.0,9928.0
15769986,1.0,28.0,0.0,7.0,20121218.0,41.0,30.0,99.0,99.0,1.0,...,1.0,28.0,41.0,1.0,129.0,129.0,34.0,5.0,48.0,9064.0
9347708,15.0,27.0,1.0,4.0,20160804.0,37.0,30.0,149.0,149.0,1.0,...,15.0,27.0,37.0,1.0,149.0,149.0,63.0,13.0,72.0,18992.0
6588668,1.0,28.0,0.0,3.0,20130504.0,38.0,30.0,149.0,149.0,0.0,...,1.0,28.0,38.0,0.0,149.0,149.0,27.0,4.0,12.0,3682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5417686,13.0,39.0,1.0,9.0,20050130.0,34.0,30.0,149.0,149.0,1.0,...,13.0,39.0,34.0,1.0,149.0,149.0,19.0,7.0,25.0,6776.0
16012604,18.0,25.0,0.0,9.0,20140904.0,33.0,30.0,149.0,149.0,1.0,...,18.0,25.0,33.0,1.0,149.0,149.0,44.0,15.0,45.0,12080.0
16225999,1.0,28.0,0.0,7.0,20131016.0,41.0,30.0,149.0,149.0,1.0,...,1.0,28.0,41.0,1.0,149.0,149.0,21.0,5.0,19.0,5348.0
11468447,13.0,23.0,2.0,3.0,20131016.0,33.0,30.0,149.0,149.0,1.0,...,13.0,23.0,33.0,1.0,149.0,149.0,44.0,15.0,48.0,11496.0


<h2> Daten aufteilen in 70% Training und 30% Test

In [21]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 160) (5910756,)
(2533182, 160) (2533182,)


In [22]:
# converting X_train to numpy array
wsdm_arr = X_train.values
wsdm_arr

array([[1.3000e+01, 3.6000e+01, 1.0000e+00, ..., 1.0000e+01, 5.8000e+01,
        1.5480e+04],
       [1.0000e+00, 2.8000e+01, 0.0000e+00, ..., 1.0000e+01, 6.6000e+01,
        1.6736e+04],
       [1.5000e+01, 2.4000e+01, 2.0000e+00, ..., 1.5000e+01, 4.6000e+01,
        1.1888e+04],
       ...,
       [1.0000e+00, 2.8000e+01, 0.0000e+00, ..., 3.0000e+00, 4.3000e+01,
        1.0304e+04],
       [1.0000e+00, 2.8000e+01, 0.0000e+00, ..., 1.3000e+01, 5.9000e+01,
        1.8896e+04],
       [1.0000e+00, 2.8000e+01, 0.0000e+00, ..., 1.4000e+01, 5.9000e+01,
        1.5952e+04]])

In [38]:
wsdm_arr.shape

(5910756, 160)

In [24]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train_incraw.npz", wsdm_arr)

In [25]:
# displaying y_train
wsdm_labels_arr = y_train
wsdm_labels_arr

[0, 0, 0, 0, 1, ..., 0, 0, 0, 0, 0]
Length: 5910756
Categories (2, int64): [0, 1]

In [26]:
wsdm_labels_arr.shape

(5910756,)

In [27]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_wsdm_labels_incraw.npz", wsdm_labels_arr)

In [28]:
# converting X_test to numpy array
X_arr = X_test.values
X_arr

array([[1.3000e+01, 2.8000e+01, 1.0000e+00, ..., 6.0000e+00, 7.4000e+01,
        1.8480e+04],
       [1.3000e+01, 2.8000e+01, 1.0000e+00, ..., 1.4000e+01, 6.1000e+01,
        1.7152e+04],
       [1.0000e+00, 2.8000e+01, 0.0000e+00, ..., 5.0000e+00, 3.9000e+01,
        9.5920e+03],
       ...,
       [1.1000e+01, 2.8000e+01, 1.0000e+00, ..., 1.3000e+01, 5.9000e+01,
        1.7600e+04],
       [1.0000e+01, 2.8000e+01, 1.0000e+00, ..., 1.0000e+01, 5.8000e+01,
        1.6096e+04],
       [6.0000e+00, 2.4000e+01, 2.0000e+00, ..., 1.1000e+01, 4.5000e+01,
        1.2200e+04]])

In [39]:
# checking shape of test array
X_arr.shape

(2533182, 160)

In [30]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_incraw.npz", X_arr)

In [31]:
# displaying y_test
X_labels_arr = y_test
X_labels_arr

[0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0]
Length: 2533182
Categories (2, int64): [0, 1]

In [32]:
# checking shape of test labels array
X_labels_arr.shape

(2533182,)

In [33]:
# saving it for further use
np.savez_compressed(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_labels_incraw.npz", X_labels_arr)

<h1> 5. Modelling

In [41]:
# loading library
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
#import seaborn as sns
#from prettytable import PrettyTable

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#import lightgbm as lgb
#import xgboost as xgb
from sklearn import metrics
#import catboost as cb


In [42]:
X_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_train_incraw.npz")
y_train = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_wsdm_labels_incraw.npz")
X_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_incraw.npz")
y_test = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\actual_test_labels_incraw.npz")

In [43]:
X_train = X_train['arr_0']
X_test = X_test['arr_0']
y_train = y_train['arr_0']
y_test = y_test['arr_0']

# getting shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5910756, 160) (5910756,)
(2533182, 160) (2533182,)


In [44]:
# loading columns
cols = np.load(r"D:\Work_Masterarbeit\wsdm\preprocessed_4\columns_incraw.npz")
# converting numpy array to python list
cols = cols['arr_0'].tolist()
# removing first two cols 'msno' and 'is_churn'
cols.remove('msno')
cols.remove('is_churn_x')

<h1> Logistic Regression

In [ ]:
%%time

# fitting logistic regression model with best parameter values
lr_model = LogisticRegression(penalty='l2', tol=0.0001, C=10, solver='newton-cg', class_weight='balanced', n_jobs=-1)
lr_model.fit(X_train, y_train)

In [ ]:
# Reference: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the trained logistic regression ML model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_lr.sav"
joblib.dump(lr_model, filename)

In [ ]:
# loading the best model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_lr.sav"
loaded_model = joblib.load(filename)

In [ ]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

<h3>Model Evaluation

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

lr_predict=lr_model.predict(X_test)
lr_predict


print('Accuracy:', accuracy_score(y_test, lr_predict))
print('F1 score weighted:', f1_score(y_test, lr_predict,average='weighted'))
print('F1 score macro:', f1_score(y_test, lr_predict, average='macro'))
print('Recall:', recall_score(y_test, lr_predict,average='weighted'))
print('Precision:', precision_score(y_test, lr_predict,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, lr_predict))
print('\n confussion matrix:\n',confusion_matrix(y_test, lr_predict))

<h2> Decision Tree

In [ ]:
%%time

# training a decision tree classifier model with best parameters
dtc_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=15, min_samples_split=2, min_samples_leaf=1)
dtc_model.fit(X_train, y_train)

In [ ]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_dtc.sav"
joblib.dump(dtc_model, filename)

In [ ]:
# loading the best decision tree classifier model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_dtc.sav"
loaded_model = joblib.load(filename)

In [ ]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))

<h3> Model Evaluation

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

dtc_modelpred=dtc_model.predict(X_test)
dtc_modelpred


print('Accuracy:', accuracy_score(y_test, dtc_modelpred))
print('F1 score weighted:', f1_score(y_test, dtc_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, dtc_modelpred, average='macro'))
print('Recall:', recall_score(y_test, dtc_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, dtc_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, dtc_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, dtc_modelpred))

<h2>Randomforest

In [ ]:
%%time

# training the random forest classifier for best hyper parameters
rf_model = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=15, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', bootstrap=True, n_jobs=-1)
rf_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

rf_modelpred=rf_model.predict(X_test)
rf_modelpred


print('Accuracy:', accuracy_score(y_test, rf_modelpred))
print('F1 score weighted:', f1_score(y_test, rf_modelpred,average='weighted'))
print('F1 score macro:', f1_score(y_test, rf_modelpred, average='macro'))
print('Recall:', recall_score(y_test, rf_modelpred,average='weighted'))
print('Precision:', precision_score(y_test, rf_modelpred,average='weighted'))
print('\n clasification report:\n', classification_report(y_test, rf_modelpred))
print('\n confussion matrix:\n',confusion_matrix(y_test, rf_modelpred))

In [ ]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/#:~:text=Saving%20Your%20Model-,Save%20Your%20Model%20with%20pickle,it%20to%20make%20new%20predictions.
# saving the decision tree classifier model
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_rf.sav"
joblib.dump(rf_model, filename)

In [ ]:
# loading the best decision tree classifier model from disk
filename = r"D:\Work_Masterarbeit\wsdm\preprocessed_4\finalized_model_rf.sav"
loaded_model = joblib.load(filename)

In [ ]:
## Logloss
# predicting probabilities for X_cv
pred_test = loaded_model.predict_proba(X_test)
# getting probabilities corresponding to class label 1 only
pred_test = pred_test[:,1]
# calculating log loss
print('Test log-loss: ', log_loss(y_test, pred_test))